In [54]:
import numpy as np
import pandas as pd

In [55]:
layerXtest = pd.read_csv('Weights2 - 6th  layer/Uk_weights2_X_test.csv',header=None).to_numpy()
layerYtest = pd.read_csv('Weights2 - 6th  layer/Uk_weights2_Y_test.csv',header=None).to_numpy()
layerXtrain = pd.read_csv('Weights2 - 6th  layer/Uk_weights2_X_validation.csv',header=None).to_numpy()
layerYtrain = pd.read_csv('Weights2 - 6th  layer/Uk_weights2_Y_validation.csv',header=None).to_numpy()
smXtest = pd.read_csv('Weights2 - Softmax/Uk_weights2_X_test.csv',header=None).to_numpy()
smYtest = pd.read_csv('Weights2 - Softmax/Uk_weights2_Y_test.csv',header=None).to_numpy()
smXtrain = pd.read_csv('Weights2 - Softmax/Uk_weights2_X_validation.csv',header=None).to_numpy()
smYtrain = pd.read_csv('Weights2 - Softmax/Uk_weights2_Y_validation.csv',header=None).to_numpy()

In [56]:
def NLPD(actual,predicted):
    """(normalised) Negative Log Predictive Density
    Definition of Negative Log Predictive Density (NLPD):
    $$L = -\frac{1}{n} \sum_{i=1}^n \log p(y_i=t_i|\mathbf{x}_i)$$
    See http://mlg.eng.cam.ac.uk/pub/pdf/QuiRasSinetal06.pdf, page 13.
    "This loss penalizes both over and under-confident predictions."
    but
    "The NLPD loss favours conservative models, that is models that tend to be under-confident
    rather than over-confident. This is illustrated in Fig. 7, and canbe deduced from the fact that
    logarithms are being used. An interesting way ofusing the NLPD is to give it relative to the NLPD
    of a predictor that ignoresthe inputs and always predicts the same Gaussian predictive distribution,
    withmean and variance the empirical mean and variance of the training data. Thisrelative NLPD
    translates into a gain of information with respect to the simpleGaussian predictor described."
    
    actual = 2d array of indices (Nx1), values are from 0 to D-1.
    predicted = 2d array of probabilities (NxD) sum to one along rows.
    """
    assert np.max(np.abs(np.sum(predicted,1)-1))<1e-5 #confirmed every row roughly adds up to one
    return -np.sum(np.log(np.take_along_axis(predicted,actual.astype(int),axis=1)))
    

# CNN solution (softmax)

In [57]:
#Deep Neural network
NLPD(smYtest,smXtest)

5655.690000928037

Trying to regularise the CNN softmax solution to see if it helps (as we regularise our EUE method, so it's fair to try it here too).

In [58]:
tempX = smXtest+0.01
tempX=(tempX.T/np.sum(tempX,1)).T

In [59]:
#Deep Neural network
NLPD(smYtest,tempX)

5764.5842905439085

No it doesn't.

# Empirical Uncertainty Estimation

Using just the confusion matrix

In [60]:
import matplotlib.pyplot as plt
%matplotlib inline

In [61]:
preds = np.argmax(smXtrain,1)
conf = np.zeros([19,19])
for p,a in zip(preds,smYtrain[:,0].astype(int)):
    conf[p,a]+=1.0
conf+=0.01
conf = (conf.T/np.sum(conf,1)).T
#plt.matshow(conf)
#plt.colorbar()
newprobs = np.zeros_like(smXtest)
preds = np.argmax(smXtest,1)
for i,p in enumerate(preds):
    newprobs[i,:] = conf[p,:]

In [62]:
NLPD(smYtest,newprobs)

5437.860361644256

Using the probabilities to build a sort of "probability confusion matrix" (this might have a name).

In [83]:
conf = np.zeros([19,19])
for x,a in zip(smXtrain,smYtrain[:,0].astype(int)):
    conf[:,a]+=x
conf+=0.05
conf = (conf.T/np.sum(conf,1)).T
#plt.matshow(conf)
#plt.colorbar()
newprobs = np.zeros_like(smXtest)
preds = np.argmax(smXtest,1)
for i,p in enumerate(preds):
    newprobs[i,:] = conf[p,:]

In [84]:
NLPD(smYtest,newprobs)

5281.081177818252

So the NLPD is reduced to 5281 from 5655 (6.6% less).

I think the benefit might be stronger in the less common speicies, it might be interesting to weight them evenly (at the moment as you mentioned it's not been rebalanced).